In [2]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch

from microgpt.model import MicroGPTConfig, MicroGPT

In [3]:
# -----------------------------------------------------------------------------
# I/O
out_dir = 'out'
eval_interval = 2000
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval

# data
dataset = 'shakespeare_char'
gradient_accumulation_steps = 5 * 8 # used to simulate larger batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = True # use PyTorch 2.0 to compile the model to be faster

def load_config():
    """Load configuration overrides from config.py"""
    try:
        # Get the directory where this script is located
        # Handle both script and notebook environments
        try:
            # For script environment
            script_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            # For notebook environment - use current working directory
            script_dir = os.getcwd()
            # If we're in a notebook, try to find the pretrain directory
            if 'pretrain' not in script_dir:
                # Look for pretrain directory in current path or parent directories
                current_dir = script_dir
                for _ in range(3):  # Look up to 3 levels up
                    if os.path.exists(os.path.join(current_dir, 'pretrain')):
                        script_dir = os.path.join(current_dir, 'pretrain')
                        break
                    parent_dir = os.path.dirname(current_dir)
                    if parent_dir == current_dir:  # Reached root
                        break
                    current_dir = parent_dir
        
        config_path = os.path.join(script_dir, 'config.py')
        
        # Read and execute the config file
        with open(config_path, 'r') as f:
            config_content = f.read()
        
        # Execute the config content in the current globals
        exec(config_content, globals())
        
        print(f"Configuration loaded from {config_path}")
        print(f"Dataset: {dataset}")
        print(f"Out dir: {out_dir}")
        print(f"Batch size: {batch_size}")
        print(f"Block size: {block_size}")
    except FileNotFoundError:
        print(f"config.py not found at {config_path}, using default configuration")
        pass  # config.py is optional

# Load configuration immediately when module is imported
load_config()

Configuration loaded from /home/wukong/git.repo/pypi/microgpt/notebook/config.py
Dataset: shakespeare_char
Out dir: out
Batch size: 64
Block size: 256


In [4]:

# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

In [5]:

# various inits, derived attributes, I/O setup
# single GPU setup
master_process = True
seed_offset = 0
ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on matmul
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader - data_dir will be set in main() after config is loaded
def get_batch(split, data_dir):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

tokens per iteration will be: 16,384


In [6]:

# init these up here
iter_num = 0
best_val_loss = 1e9

# model init - always initialize a new model from scratch
print("Initializing a new model from scratch")
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=50304, dropout=dropout) # start with model_args from command line
gptconf = MicroGPTConfig(**model_args)
model = MicroGPT(gptconf)

# crop down the model block size if desired, using model surgery
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.amp.GradScaler('cuda', enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

Initializing a new model from scratch
number of parameters: 29.94M
num decayed parameter tensors: 26, with 30,031,872 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True
compiling the model... (takes a ~minute)


In [8]:
from hiq.vis import print_model
print_model(model)


🌳 OptimizedModule<all params:30036864>
└── MicroGPT(_orig_mod)
    ├── ModuleDict(transformer)
    │   ├── Embedding(wte)|weight[50304,384]
    │   ├── Embedding(wpe)|weight[256,384]
    │   ├── ModuleList(h)
    │   │   └── 💠 ]8;id=569227;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\Block]8;;\]8;id=615546;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(0-5)<🦜:1770240x6>]8;;\
    │   │       ┣━━ 💠 ]8;id=118570;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\LayerNorm]8;;\]8;id=420989;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(ln_1,ln_2)<🦜:384x2>|]8;;\]8;id=157852;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\weight[384]]8;;\
    │   │       ┣━━ CausalSelfAttention(attn)
    │   │       ┃   ┣━━ Linear(c_attn)|weight[1152,384]
    │   │       ┃   ┗━━ Linear(c_proj)|weight[384,384]
    │   │       ┗━━ MLP(mlp)
    │   │           ┣━━ Linear(c_fc)|weight[1536,384]
    │   │           ┗━━ Linear(c_proj)|weight[384,1536]
    │   └── LayerNorm(ln_f)|weight[384]
    └── Linear(lm_head)|weight[50304,384]

In [9]:

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss(data_dir):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, data_dir)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [11]:
def main():
    global iter_num, best_val_loss, model, optimizer
    
    # training loop
    # data_dir will be set here after config is loaded
    data_dir = os.path.join('data', dataset)
    
    # attempt to derive vocab_size from the dataset
    meta_path = os.path.join(data_dir, 'meta.pkl')
    meta_vocab_size = None
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        meta_vocab_size = meta['vocab_size']
        print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")
    
    # update model_args with vocab_size if found
    if meta_vocab_size is not None:
        model_args['vocab_size'] = meta_vocab_size
        # recreate the model with the correct vocab_size
        gptconf = MicroGPTConfig(**model_args)
        model = MicroGPT(gptconf)
        model.to(device)
        # recompile if needed
        if compile:
            print("recompiling the model with updated vocab_size...")
            unoptimized_model = model
            model = torch.compile(model)
        # reconfigure optimizer
        optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
    
    # Print the final model structure after all potential modifications
    from hiq.vis import print_model
    print_model(model)
    
    X, Y = get_batch('train', data_dir) # fetch the very first batch
    t0 = time.time()
    local_iter_num = 0 # number of iterations in the lifetime of this process
    raw_model = model # no DDP wrapper needed
    running_mfu = -1.0
    while True:

        # determine and set the learning rate for this iteration
        lr = get_lr(iter_num) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # evaluate the loss on train/val sets and write checkpoints
        if iter_num % eval_interval == 0 and master_process:
            losses = estimate_loss(data_dir)
            print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

            if losses['val'] < best_val_loss or always_save_checkpoint:
                best_val_loss = losses['val']
                if iter_num > 0:
                    checkpoint = {
                        'model': raw_model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'model_args': model_args,
                        'iter_num': iter_num,
                        'best_val_loss': best_val_loss,
                        'config': config,
                    }
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
        if iter_num == 0 and eval_only:
            break

        # forward backward update, with optional gradient accumulation to simulate larger batch size
        # and using the GradScaler if data type is float16
        for micro_step in range(gradient_accumulation_steps):
            with ctx:
                logits, loss = model(X, Y)
                loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            X, Y = get_batch('train', data_dir)
            # backward pass, with gradient scaling if training in fp16
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        # step the optimizer and scaler if training in fp16
        scaler.step(optimizer)
        scaler.update()
        # flush the gradients as soon as we can, no need for this memory anymore
        optimizer.zero_grad(set_to_none=True)

        # timing and logging
        t1 = time.time()
        dt = t1 - t0
        t0 = t1
        if iter_num % log_interval == 0 and master_process:
            # get loss as float. note: this is a CPU-GPU sync point
            # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
            lossf = loss.item() * gradient_accumulation_steps
            if local_iter_num >= 5: # let the training loop settle a bit
                mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
                running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
            print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
        iter_num += 1
        local_iter_num += 1

        # termination conditions
        if iter_num > max_iters:
            break

In [13]:
!python -m microgpt.prepare_dataset

Saving processed data to: /home/wukong/git.repo/pypi/microgpt/notebook/data/shakespeare_char
length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [14]:
main()

found vocab_size = 65 (inside data/shakespeare_char/meta.pkl)
number of parameters: 10.65M
recompiling the model with updated vocab_size...
num decayed parameter tensors: 26, with 10,740,096 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True


🌳 OptimizedModule<all params:10745088>
└── MicroGPT(_orig_mod)
    ├── ModuleDict(transformer)
    │   ├── Embedding(wte)|weight[65,384]
    │   ├── Embedding(wpe)|weight[256,384]
    │   ├── ModuleList(h)
    │   │   └── 💠 ]8;id=109338;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\Block]8;;\]8;id=477708;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(0-5)<🦜:1770240x6>]8;;\
    │   │       ┣━━ 💠 ]8;id=435315;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\LayerNorm]8;;\]8;id=377079;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\(ln_1,ln_2)<🦜:384x2>|]8;;\]8;id=517943;/home/wukong/git.repo/pypi/microgpt/microgpt/model.py\weight[384]]8;;\
    │   │       ┣━━ CausalSelfAttention(attn)
    │   │       ┃   ┣━━ Linear(c_attn)|weight[1152,384]
    │   │       ┃   ┗━━ Linear(c_proj)|weight[384,384]
    │   │       ┗━━ MLP(mlp)
    │   │           ┣━━ Linear(c_fc)|weight[1536,384]
    │   │           ┗━━ Linear(c_proj)|weight[384,1536]
    │   └── LayerNorm(ln_f)|weight[384]
    └── Linear(lm_head)|weight[65,384]

step 0: train loss 4.3128, val loss 4.3080
iter 0: loss 4.2885, time 17601.62ms, mfu -100.00%
iter 10: loss 3.2350, time 15.02ms, mfu 24.80%
iter 20: loss 2.8026, time 15.46ms, mfu 24.73%
iter 30: loss 2.6264, time 15.08ms, mfu 24.73%
iter 40: loss 2.5543, time 14.43ms, mfu 24.84%
iter 50: loss 2.5380, time 15.37ms, mfu 24.78%
iter 60: loss 2.5118, time 15.82ms, mfu 24.66%
iter 70: loss 2.5038, time 15.27ms, mfu 24.63%
iter 80: loss 2.4857, time 14.56ms, mfu 24.73%
iter 90: loss 2.4883, time 15.26ms, mfu 24.70%
iter 100: loss 2.4635, time 15.46ms, mfu 24.64%
iter 110: loss 2.4543, time 15.25ms, mfu 24.62%
iter 120: loss 2.4165, time 14.49ms, mfu 24.73%
iter 130: loss 2.4358, time 15.25ms, mfu 24.70%
iter 140: loss 2.4238, time 15.66ms, mfu 24.61%
iter 150: loss 2.3916, time 15.22ms, mfu 24.60%
iter 160: loss 2.3452, time 14.51ms, mfu 24.70%
iter 170: loss 2.3514, time 15.52ms, mfu 24.63%
iter 180: loss 2.3390, time 15.89ms, mfu 24.52%
iter 190: loss 2.2973, time 15.05ms, mfu 24.54%
ite

In [16]:
!python -m microgpt.sample

Loaded configuration from: /home/wukong/git.repo/pypi/microgpt/microgpt/pretrain/config.py
Using out_dir: out
Loading checkpoint from: out/ckpt.pt
number of parameters: 10.65M
Using data directory meta.pkl from: data/shakespeare_char/meta.pkl
Loading meta from data/shakespeare_char/meta.pkl...


All the bride with the sweet made of black and mage-day
Beat the supposities of a bed,
And that are away with himself thousand of spite,
Making that he will be mine, send in the state,
Give thee the noble benefit. Ah, no!
Thou liest me with a special place, and to see thy face!
I will not see the king Willow be Caius Marcius down.

Third Messenger:
A life that known not the sorrow is not to thy flesh than fear.

MENENIUS:
Thou art too discharged with a helperous court
and no very poor court. My
---------------

Menenius, and grave we would that
The celest of your presence.

CORIOLANUS:
What is the country?

MENENIUS:
Mark you, lords, is more than the sea
In aught of the madness.

CORIOLANUS:
Ma